<a href="https://colab.research.google.com/github/serhatataman/ProjectAmbitionColab/blob/main/Project_Ambition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Ambition



## Setup

Check the NVIDIA driver's availability
Note: if this step gives `NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running` error, select GPU as hardware accelerator in `Edit > Notebook settings`.

In [31]:
!nvidia-smi

Sat Jan  1 20:23:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Mount to the drive to the following location.

```
/content/drive/MyDrive/data
```

Use ```ls``` command to establish the exact path for your images.

In [10]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Using Google CoLab")
except:
    print("Not using Google CoLab")
    COLAB = False

Mounted at /content/drive
Using Google CoLab


In [11]:
!ls /content/drive/MyDrive

'Colab Notebooks'			  'Master Application Docs.'
'Copy of [#1 Sharing Session] TSO.docx'    Personal
'Copy of [#4 Re-Entry Session] TSO.docx'   ProjectAmbition
'CS:GO config'				  'Reflection groups guideline.gdoc'
 CV					   UDI
'Graduation Thesis'			   Wallpapers
 ielts



### Import NVIDIA stylegan3

If the repo is already installed, it will skip the installation process and change into the repo’s directory. If not, it will install all the files necessary.
Also, create `downloads` and `datasets` folders.

In [12]:
import os
if os.path.isdir("/content/drive/MyDrive/ProjectAmbition"):
    %cd "/content/drive/MyDrive/ProjectAmbition"
else:
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir ProjectAmbition
    %cd ProjectAmbition
    !git clone https://github.com/NVlabs/stylegan3
    !mkdir downloads
    !mkdir datasets
    !mkdir raw_data

/content/drive/MyDrive/ProjectAmbition


In [13]:
%cd "/content/drive/MyDrive/ProjectAmbition/stylegan3"
!git config --global user.name "serhatataman"
!git config --global user.email "serhatataman13@hotmail.com"
!git fetch origin
!git checkout origin/main -- train.py

/content/drive/MyDrive/ProjectAmbition/stylegan3


In [14]:
!pip install ninja

     |████████████████████████████████| 108 kB 5.1 MB/s 



### Checking if directories/files exist

In [15]:
import os

if not os.path.exists("/content/drive/MyDrive/ProjectAmbition/stylegan3/dataset_tool.py"):
  raise FileNotFoundError("dataset_tool.py file does not exist!")

if not os.path.exists("/content/drive/MyDrive/ProjectAmbition/raw_data"):
  raise FileNotFoundError("raw_data folder does not exist!")

if not os.path.exists("/content/drive/MyDrive/ProjectAmbition/dataset"):
  print("dataset folder does not exist! creating the folder...")
  os.mkdir("/content/drive/MyDrive/ProjectAmbition/dataset")

if not os.path.exists("/content/drive/MyDrive/ProjectAmbition/output"):
  print("output folder does not exist! creating the folder...")
  os.mkdir("/content/drive/MyDrive/ProjectAmbition/output")

dataset folder does not exist! creating the folder...
output folder does not exist! creating the folder...


## Cleanup images

See what dataset_tool.py can do

In [ ]:
cmd = f"/usr/bin/python3 /content/drive/MyDrive/ProjectAmbition/stylegan3/dataset_tool.py --help"
!{cmd}

### Remove frames

Remove frames if there are any. This process overrides the picture with frames removed.

In [16]:
import os
import numpy as np
from PIL import Image
from scipy.spatial import distance

# This python script removes pictures' frames if there are any

from_path = '/content/drive/MyDrive/ProjectAmbition/raw_data/'
to_path = '/content/drive/MyDrive/ProjectAmbition/raw_data/'


def find_left():
    left = 0
    for i in range(0, w_pad):
        r_stdev = np.std(np_img[h_pad:-h_pad, i:i + 1, 0:1])
        g_stdev = np.std(np_img[h_pad:-h_pad, i:i + 1, 1:2])
        b_stdev = np.std(np_img[h_pad:-h_pad, i:i + 1, 2:3])
        if r_stdev * r_stdev + g_stdev * g_stdev + b_stdev * b_stdev > thresh1:
            break

        r_med = np.median(np_img[h_pad:-h_pad, i:i + 1, 0:1])
        g_med = np.median(np_img[h_pad:-h_pad, i:i + 1, 1:2])
        b_med = np.median(np_img[h_pad:-h_pad, i:i + 1, 2:3])
        dst = distance.euclidean((r_med, g_med, b_med), (r_global_med, g_global_med, b_global_med))
        if dst < thresh2:
            break

        left = left + 1
    return left


def find_top():
    top = 0
    for i in range(0, h_pad):
        r_stdev = np.std(np_img[i:i + 1, w_pad:-w_pad, 0:1])
        g_stdev = np.std(np_img[i:i + 1, w_pad:-w_pad, 1:2])
        b_stdev = np.std(np_img[i:i + 1, w_pad:-w_pad, 2:3])
        if r_stdev * r_stdev + g_stdev * g_stdev + b_stdev * b_stdev > thresh1:
            break

        r_med = np.median(np_img[i:i + 1, w_pad:-w_pad, 0:1])
        g_med = np.median(np_img[i:i + 1, w_pad:-w_pad, 1:2])
        b_med = np.median(np_img[i:i + 1, w_pad:-w_pad, 2:3])
        dst = distance.euclidean((r_med, g_med, b_med), (r_global_med, g_global_med, b_global_med))
        if dst < thresh2:
            break

        top = top + 1
    return top


def find_right(right):
    right = w
    for i in range(0, w_pad):
        r_stdev = np.std(np_img[h_pad:-h_pad, w - i - 1:w - i, 0:1])
        g_stdev = np.std(np_img[h_pad:-h_pad, w - i - 1:w - i, 1:2])
        b_stdev = np.std(np_img[h_pad:-h_pad, w - i - 1:w - i, 2:3])
        if r_stdev * r_stdev + g_stdev * g_stdev + b_stdev * b_stdev > thresh1:
            break

        r_med = np.median(np_img[h_pad:-h_pad, w - i - 1:w - i, 0:1])
        g_med = np.median(np_img[h_pad:-h_pad, w - i - 1:w - i, 1:2])
        b_med = np.median(np_img[h_pad:-h_pad, w - i - 1:w - i, 2:3])
        dst = distance.euclidean((r_med, g_med, b_med), (r_global_med, g_global_med, b_global_med))
        if dst < thresh2:
            break

        right = right - 1
    return right


def find_bottom(bottom):
    for i in range(0, h_pad):
        r_stdev = np.std(np_img[h - i - 1:h - i, w_pad:-w_pad, 0:1])
        g_stdev = np.std(np_img[h - i - 1:h - i, w_pad:-w_pad, 1:2])
        b_stdev = np.std(np_img[h - i - 1:h - i, w_pad:-w_pad, 2:3])
        if r_stdev * r_stdev + g_stdev * g_stdev + b_stdev * b_stdev > thresh1:
            break

        r_med = np.median(np_img[h - i - 1:h - i, w_pad:-w_pad, 0:1])
        g_med = np.median(np_img[h - i - 1:h - i, w_pad:-w_pad, 1:2])
        b_med = np.median(np_img[h - i - 1:h - i, w_pad:-w_pad, 2:3])
        dst = distance.euclidean((r_med, g_med, b_med), (r_global_med, g_global_med, b_global_med))
        if dst < thresh2:
            break

        bottom = bottom - 1
    return bottom


for file in os.listdir(from_path):
    path = os.path.join(from_path, file)
    img = Image.open(path)
    file_name, file_extension = os.path.splitext(path)
    print('Removing frames for image:', file_name + file_extension)

    np_img = np.asarray(img)
    # print("shape = " + str(np_img.shape))

    thresh1 = 15000
    thresh2 = 30
    w = img.width
    h = img.height
    pad = 30
    w_pad = w // pad
    h_pad = h // pad

    r_global_med = np.median(np_img[h_pad:-h_pad, w_pad:-w_pad, 0:1])
    g_global_med = np.median(np_img[h_pad:-h_pad, w_pad:-w_pad, 1:2])
    b_global_med = np.median(np_img[h_pad:-h_pad, w_pad:-w_pad, 2:3])

    left = find_left()
    top = find_top()
    right = find_right(w)
    bottom = find_bottom(h)

    # print("left = " + str(left) + ", top = " + str(top) +
      #    ", right = " + str(right) + ", bottom = " + str(bottom) + "\n")

    # img.save(to_path + file)  # save the original
    cropped_img = img.crop((left, top, right, bottom))
    cropped_img.save(file_name + file_extension)  # and the cropped version


Removing frames for image: /content/drive/MyDrive/ProjectAmbition/raw_data/peasant-and-horse-1910.jpg
Removing frames for image: /content/drive/MyDrive/ProjectAmbition/raw_data/horses.jpg
Removing frames for image: /content/drive/MyDrive/ProjectAmbition/raw_data/spring-1907.jpg
Removing frames for image: /content/drive/MyDrive/ProjectAmbition/raw_data/the-rise-of-green-square-and-the-woman-s-violin-1916.jpg


### Resize images

Resize images to 1024x1024 and override them.

In [26]:
import os
from PIL import Image

image_source_path = '/content/drive/MyDrive/ProjectAmbition/raw_data/'
image_target_path = '/content/drive/MyDrive/ProjectAmbition/raw_data/'

# We are using Pillow to resize all images to our desired size

for filename in os.listdir(image_source_path):
    path = os.path.join(image_source_path, filename)
    image = Image.open(path).resize((1024, 1024), Image.ANTIALIAS)

    resized_image = image.save(image_target_path + filename)
    print(f"{filename} image is resized...")


peasant-and-horse-1910.jpg image is resized...
horses.jpg image is resized...
spring-1907.jpg image is resized...
the-rise-of-green-square-and-the-woman-s-violin-1916.jpg image is resized...


## Convert images

Convert raw data images to dataset by using StyleGAN3's built-in dataset_tool.py

In [32]:
!python /content/drive/MyDrive/ProjectAmbition/stylegan3/dataset_tool.py --source /content/drive/MyDrive/ProjectAmbition/raw_data/ --dest /content/drive/MyDrive/ProjectAmbition/dataset/ --resolution=1024x1024 --transform=center-crop-wide

100% 4/4 [00:00<00:00,  8.77it/s]


The following command can be used to clear out the newly created dataset. If something goes wrong and you need to clean up your images and rerun the above command, you should delete your partially created dataset directory.

In [25]:
# !rm -R /content/drive/MyDrive/ProjectAmbition/dataset/*

## Training

### Initial training

In [ ]:
cmd = f"/usr/bin/python3 /content/drive/MyDrive/ProjectAmbition/stylegan3/train.py --help"
!{cmd}

In [33]:
import os

!export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128

# Modify these to suit your needs
OUTPUT = "/content/drive/MyDrive/ProjectAmbition/output"
DATA = "/content/drive/MyDrive/ProjectAmbition/dataset"
# Snap: How often should the model generate samples and a .pkl file
SNAP = 4
# Mirrored: Should the images be mirrored left to right?
MIRRORED = True


# Build the command and run it
cmd = f"/usr/bin/python3 /content/drive/MyDrive/ProjectAmbition/stylegan3/train.py --cfg=stylegan3-t --gpus=1 --batch=32 --gamma=8.2 --snap {SNAP} --outdir {OUTPUT} --data {DATA} --mirror={MIRRORED}"
!{cmd}


Training options:
{
  "G_kwargs": {
    "class_name": "training.networks_stylegan3.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "channel_base": 32768,
    "channel_max": 512,
    "magnitude_ema_beta": 0.9988915792636801
  },
  "D_kwargs": {
    "class_name": "training.networks_stylegan2.Discriminator",
    "block_kwargs": {
      "freeze_layers": 0
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 32768,
    "channel_max": 512
  },
  "G_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.0025
  },
  "D_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.002
  },
  "loss_kwargs": {
    "class_name": "training.loss.StyleGAN2Loss",
    "r1_gamma": 8.2
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "prefetch_fact

### Resume training

In [ ]:
import os


# Modify these to suit your needs
OUTPUT = "/content/drive/MyDrive/ProjectAmbition/output"
NETWORK = "network-snapshot-000100.pkl"
RESUME = os.path.join(OUTPUT, "00008-circuit-auto1-resumecustom", NETWORK)
DATA = "/content/drive/MyDrive/ProjectAmbition/dataset"
# Snap: How often should the model generate samples and a .pkl file
SNAP = 4
# Mirrored: Should the images be mirrored left to right?
MIRRORED = True
# Mirrored_y: Should the images be mirrored top to bottom?
MIRRORED_Y = True


# Build the command and run it
cmd = f"/usr/bin/python3 /content/drive/MyDrive/ProjectAmbition/stylegan3/train.py --cfg=stylegan3-t --gpus=8 --batch=32 --gamma=8.2 --snap {SNAP} --resume {RESUME} --outdir {OUTPUT} --data {DATA} --mirror={MIRRORED} --mirrory={MIRRORED_Y}"
!{cmd}

## Generate image